In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from prophet.plot import plot_plotly, plot_components_plotly
from prophet import Prophet
from statsmodels.tools.eval_measures import rmse

df = pd.read_csv('/Users/alanchan/Documents/UCL/Year 4/ELEC0088/SNS assignment 22-23/London Property Price/ukhpi-property-type-london-from-1977-06-01-to-2018-06-01.csv')
print(df.info())
df.dropna(inplace= True)
df.reset_index(drop=True, inplace=True)
print(df.info())

In [ ]:
# print(df.tail())
df = df.rename(columns={'Average price All property types': 'y', 'Period':'ds'})
df['ds'] = pd.to_datetime(df['ds'], format = '%Y-%m')
print(df.tail())

In [ ]:
df.sort_values(by='ds', inplace = True)
df.reset_index(drop=True, inplace=True)
# print(df.head())
print(df.tail())

In [ ]:
df.plot(x='ds',y='y',figsize=(18,6))
print(len(df))

In [ ]:
Test_split = 12
train = df.iloc[:len(df)-Test_split]
test = df.iloc[len(df)-Test_split:]

print(len(train))
print(len(test))

print(df.tail())

In [ ]:
def preprocessing (multi_var,periods):
    t = train[['ds',multi_var]]
    t = t.rename(columns={multi_var: 'y', 'date':'ds'})
    model = Prophet()
    model.fit(t)
    future = model.make_future_dataframe(periods=periods)
    forecast = model.predict(future)
    return forecast['yhat']

periods = 12*2

Sales_volume = preprocessing('Sales volume',periods)
House_price_index_All_property_types = preprocessing('House price index All property types',periods)
Percentage_change_monthly_All_property_types = preprocessing('Percentage change (monthly) All property types',periods)
Percentage_change_yearly_All_property_types = preprocessing('Percentage change (yearly) All property types',periods)

In [ ]:
m = Prophet()
m.add_regressor('Sales volume')
m.add_regressor('House price index All property types')
m.add_regressor('Percentage change (monthly) All property types')
m.add_regressor('Percentage change (yearly) All property types')
m.fit(train)
future = m.make_future_dataframe(periods=periods, freq = 'MS')
future['Sales volume'] = Sales_volume
future['House price index All property types'] = House_price_index_All_property_types
future['Percentage change (monthly) All property types'] = Percentage_change_monthly_All_property_types
future['Percentage change (yearly) All property types'] = Percentage_change_yearly_All_property_types


In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
plot_plotly(m ,forecast)

In [ ]:
plot_components_plotly(m, forecast)

In [ ]:
predictions = forecast.iloc[len(train):len(train)+len(test)]['yhat']
print("Root Mean Squared Error between actual and  predicted values: ",rmse(predictions,test['y']))
print("Mean Value of Test Dataset:", test['y'].mean())

In [ ]:
training = forecast.iloc[:len(train)]['yhat']
print("Root Mean Squared Error between actual and  predicted values: ",rmse(training,train['y']))